
# Challenge de Reproducibilidad: MR-DBSCAN
## Algoritmo Paralelo de Clustering por Densidad usando MapReduce

---

## Información del Proyecto

**Paper:** MR-DBSCAN: An Efficient Parallel Density-based Clustering Algorithm using MapReduce  
**Autores:** Yaobin He, Haoyu Tan, Wuman Luo, et al. (Shenzhen Institutes of Advanced Technology)  
**Dataset:** 1.9 billion GPS records (Shanghai taxi data) o sintético  
**Plataforma:** Hadoop + PySpark  

**Objetivo:** Reproducir el algoritmo MR-DBSCAN en 4 etapas usando MapReduce para clustering de datos espaciales a gran escala.

---

## Parámetros del Algoritmo

- **Eps (epsilon):** Radio máximo de vecindad
  - Limpieza de datos: 0.002 (elimina ruido/outliers)
  - Identificación de regiones densas: 0.0002 (clusters más específicos)

- **MinPts:** Número mínimo de puntos para formar un cluster
  - Limpieza: 1000 puntos
  - Regiones densas: 100 puntos

- **Particiones:** Grid-based partitioning
  - Strips por dimensión: 80-160 según Eps

---

## División de Tareas por Persona

| Rol | Responsabilidades |
|-----|-------------------|
| **Persona 1** | Etapa 1 (Preprocesamiento y Particionado) + Setup inicial |
| **Persona 2** | Etapa 2 (DBSCAN Local) + Etapa 3 (Detección de Cruces) |
| **Persona 3** | Etapa 4 (Fusión Global) + Validación y Reportes |

---

## Fases del Algoritmo MR-DBSCAN

El algoritmo se ejecuta en 4 etapas usando MapReduce:

1. **Stage 1 - Preprocesamiento:** Análisis del dataset y particionamiento en grid
2. **Stage 2 - DBSCAN Local:** Clustering independiente en cada partición
3. **Stage 3 - Detección de Cruces:** Identificar clusters que cruzan fronteras (MC Sets)
4. **Stage 4 - Fusión Global:** Unificar clusters y relabeling global



---

# PARTE 1: SETUP INICIAL Y CARGA DE DATOS
## Responsable: Jose

### Paso 1.1: Configuración del Entorno Spark

**¿Qué hace este paso?**

Este paso inicializa una sesión de Spark (SparkSession), que es el punto de entrada principal para trabajar con DataFrames y RDDs en PySpark. La sesión establece la conexión con el cluster Hadoop y configura el nombre de la aplicación para identificarla en el Hadoop NameNode.

**Detalles técnicos:**
- `SparkSession.builder` permite crear una nueva sesión de forma fluida
- `.appName()` asigna el nombre "MR-DBSCAN" que verás en la interfaz Hadoop
- `.getOrCreate()` obtiene una sesión existente o crea una nueva

**¿Por qué es importante?**
- Necesario para distribuir datos y código a los nodos del cluster
- Permite usar funciones paralelas de Spark en todo el código



In [ ]:

# Inicializar SparkSession
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

def get_spark_session(app_name='MR-DBSCAN'):
    """
    Crea una nueva sesión de Spark o retorna la existente.

    Parámetros:
        app_name (str): Nombre de la aplicación para identificar en Hadoop

    Retorna:
        SparkSession: Sesión de Spark lista para usar
    """
    spark = SparkSession.builder \
        .appName(app_name) \
        .config("spark.sql.shuffle.partitions", "200") \
        .getOrCreate()

    # Configurar nivel de logging para reducir mensajes
    spark.sparkContext.setLogLevel("WARN")

    return spark

# Crear sesión
spark = get_spark_session()
print(f"✓ Sesión de Spark creada exitosamente")
print(f"  - Spark Version: {spark.version}")
print(f"  - Master: {spark.sparkContext.master()}")



### 📌 Paso 1.2: Cargar y Preprocesar los Datos

**¿Qué hace este paso?**

Este paso:
1. **Carga o genera datos:** Si tienes un archivo CSV con GPS, lo carga. Si no, genera datos sintéticos que simulan coordenadas GPS (latitud y longitud)
2. **Normalización:** Transforma las coordenadas a un rango estándar (0-1) para facilitar el particionado
3. **Estructuras de datos:** Convierte los datos a un DataFrame de Pandas (para pre-procesamiento local) o Spark (para procesamiento distribuido)

**¿Por qué es importante?**
- El MR-DBSCAN requiere datos espaciales (lat, lon) en formato limpio
- La normalización es crucial para que el grid partitioning funcione correctamente
- Los datos limpios y bien estructurados son la base del algoritmo

**Datos de ejemplo:**
- Dataset original: 1.9 mil millones de GPS del taxi de Shanghai
- Para este ejemplo: 100,000 puntos sintéticos distribuidos aleatoriamente



In [ ]:

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Opción 1: Si tienes un CSV con datos GPS, descomenta y modifica:
# data = pd.read_csv('ruta/a/tu/archivo.csv')
# data = data[['lon', 'lat']].copy()

# Opción 2: Generar datos sintéticos (para demostración)
n_points = 100000  # Número de puntos a generar
np.random.seed(42)  # Para reproducibilidad

# Simular coordenadas GPS de Shanghai (rango real)
data = pd.DataFrame({
    'lon': np.random.uniform(121.0, 122.0, size=n_points),  # Longitud
    'lat': np.random.uniform(30.9, 31.5, size=n_points)    # Latitud
})

# Normalizar coordenadas al rango [0, 1]
data['lon_norm'] = (data['lon'] - data['lon'].min()) / (data['lon'].max() - data['lon'].min())
data['lat_norm'] = (data['lat'] - data['lat'].min()) / (data['lat'].max() - data['lat'].min())

print(f"✓ Datos cargados exitosamente")
print(f"  - Número de puntos: {len(data):,}")
print(f"  - Rango Longitud: [{data['lon'].min():.6f}, {data['lon'].max():.6f}]")
print(f"  - Rango Latitud: [{data['lat'].min():.6f}, {data['lat'].max():.6f}]")
print(f"\nPrimeros 5 registros:")
print(data.head())



---

# PARTE 2: PARTICIONADO EN GRID
## Responsable: Jose

### Paso 2.1: División del Dominio Espacial con Grid

**¿Qué hace este paso?**

Este es el corazón del Stage 1 del MR-DBSCAN. El particionamiento en grid:

1. **Divide el espacio:** Crea una malla 2D que cubre todo el área geográfica
2. **Asigna particiones:** Cada punto GPS recibe un ID de partición según su ubicación en la malla
3. **Equilibra carga:** Intenta que cada partición tenga aproximadamente la misma cantidad de datos

**Conceptos clave:**
- **n_strips:** Número de divisiones por dimensión (80-160 según el paper)
  - Más strips = particiones más pequeñas (mejor balance pero más réplicas en bordes)
  - Menos strips = particiones más grandes (menos comunicación pero desbalance)
- **lon_bin / lat_bin:** Índice de la celda en cada dimensión
- **partition_id:** Identificador único de la partición (combinación de lon_bin y lat_bin)

**¿Por qué es importante?**
- MapReduce distribuye datos a diferentes nodos según la partición
- El balance de carga afecta directamente el rendimiento paralelo
- Un particionamiento pobre causa "data skew" (algunos nodos reciben más datos)

**Ejemplo visual:**
```
Si n_strips=3, espacio 0-1 se divide en:
[0-0.33, 0.33-0.67, 0.67-1.0]

Un punto en (0.5, 0.8) recibe:
lon_bin = 1 (está en [0.33-0.67])
lat_bin = 2 (está en [0.67-1.0])
partition_id = "1_2"
```



In [ ]:

# Definir parámetros de particionamiento
n_strips = 120  # Óptimo según el paper para Eps grande (0.002)

# Crear bins para dividir el espacio normalizado [0, 1]
lon_bins = np.linspace(0, 1, n_strips + 1)
lat_bins = np.linspace(0, 1, n_strips + 1)

# Asignar cada punto a su bin (celda del grid)
data['lon_bin'] = np.digitize(data['lon_norm'], lon_bins) - 1
data['lat_bin'] = np.digitize(data['lat_norm'], lat_bins) - 1

# Crear identificador único de partición
data['partition_id'] = data['lon_bin'].astype(str) + '_' + data['lat_bin'].astype(str)

# Información del particionado
n_partitions = len(data['partition_id'].unique())
print(f"✓ Particionamiento completado")
print(f"  - Número de strips por dimensión: {n_strips}")
print(f"  - Total de particiones: {n_partitions}")
print(f"  - Puntos por partición (promedio): {len(data) / n_partitions:.1f}")
print(f"  - Particiones con datos: {(data['partition_id'].value_counts() > 0).sum()}")

# Mostrar distribución de datos entre particiones
print(f"\nDistribución de datos:")
partition_counts = data['partition_id'].value_counts()
print(f"  - Partición con más datos: {partition_counts.max()} puntos")
print(f"  - Partición con menos datos: {partition_counts.min()} puntos")
print(f"  - Desviación estándar: {partition_counts.std():.1f}")



### Paso 2.2: Preparación de Datos para MapReduce

**¿Qué hace este paso?**

Prepara los datos para ser procesados en paralelo en Hadoop:

1. **Agrupa por partición:** Organiza todos los puntos que pertenecen a la misma partición
2. **Convierte a Spark DataFrame:** Transforma los datos de Pandas a Spark para procesamiento distribuido
3. **Repartición:** Distribuye las particiones entre los nodos del cluster Hadoop

**¿Por qué es importante?**
- MapReduce requiere datos en formato distribuido
- Spark DataFrame permite operaciones paralelas automáticas
- La repartición asegura que cada nodo reciba trabajo similar



In [ ]:

# Opción 1: Mantener en Pandas para procesamiento local
partition_groups = dict(list(data.groupby('partition_id')))
print(f"✓ Datos agrupados por partición")
print(f"  - Número de grupos: {len(partition_groups)}")
print(f"  - Ejemplo de keys: {list(partition_groups.keys())[:5]}")

# Opción 2: Convertir a Spark DataFrame para procesamiento distribuido
spark_df = spark.createDataFrame(data)
spark_df = spark_df.repartition('partition_id')  # Distribuir por partition_id

print(f"\n✓ Spark DataFrame creado")
print(f"  - Número de particiones Spark: {spark_df.rdd.getNumPartitions()}")
print(f"  - Total de registros: {spark_df.count():,}")
print(f"\nEstructura de datos:")
spark_df.printSchema()
